In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from ast import literal_eval
import re
from scipy import stats
from random import sample
#import scipy.stats as stats
import random
import re
import statistics
from scipy.stats import wilcoxon
from sklearn.metrics import roc_curve, auc

In [2]:
def HLA_cd8_converter(x):
    #define format of datetime
    return x.replace("[","").replace("]","").replace(",", "").replace("'","").split(" ")

def cdr3_lst_converter(x):
    #define format of datetime
    return x.replace("[","").replace("]","").replace("'","").split(" ")

def epitope_converter(x):
    #define format of datetime
    return [y for y in x.replace("[","").replace("]","").replace("\n","").split("'") if (y != '') & (y != ' ')]

def peptide_hla_converter(x):
    return re.findall("\w+\s{1}\w{1}\d+", x.replace("[","").replace("]","").replace("\n","").replace("'",""))

def literal_converter(val):
    # replace NaN with '' and perform literal eval on the rest
    return [] if val == '' else literal_eval(val)

converters = {'peptide_HLA_lst': peptide_hla_converter,
              'umi_count_lst_mhc': literal_eval,
              'umi_count_lst_TRA': literal_converter,'umi_count_lst_TRB': literal_converter,
              'cdr3_lst_TRA': cdr3_lst_converter,
              'cdr3_lst_TRB': cdr3_lst_converter,
              'HLA_lst_mhc': cdr3_lst_converter,'HLA_cd8': HLA_cd8_converter} #

In [3]:
def wilcoxon_test(intra, inter):
    w, p = wilcoxon(np.array(intra)-np.array(inter), alternative='greater')
    if (p < 0.05) & (len(intra) > 9):
        return {'test':True, 'pvalue':p}
    else:
        return {'test':False, 'pvalue':p}

# INPUT

In [5]:
SIM_INPUT = '../../experiments/exp09/run1/cat/similarity_assessment/plt_df/sim.indv.csv' #snakemake.input.sim

# Load

In [6]:
test_df = pd.read_csv(SIM_INPUT).dropna()

In [31]:
labels = test_df.filtering.unique()
print(labels)
#palette = auc_df.set_index('filtering').palette.to_dict()

['total' 'optimal threshold' 'matching HLA' 'complete TCRs'
 'specificity multiplets' 'is cell (GEX)' 'is viable cell']


In [8]:
test_df[test_df.plateau=='intra']

,plateau,rnd_sample,filtering,peptide,ct,score,cdr3s,label,AUC,AUC 0.1
0,intra,0,total,VTEHDTLLY A0101,1.0,1.792556,"('CAASQNEKLTF', 'CASRIGAAGNSPLHF')",2,0.524409,0.063776
1,intra,0,total,VTEHDTLLY A0101,5.0,1.767455,"('CAVEDRLDNDMRF', 'CSARGLGQAKEYGYTF')",2,0.524409,0.063776
2,intra,0,total,VTEHDTLLY A0101,4.0,1.968580,"('CAVTAPAYSGAGSYQLTF', 'CASSYTPGSTYEQYF')",2,0.524409,0.063776
4,intra,0,total,VTEHDTLLY A0101,437.0,1.569405,"('CAVTYSPPPKLLF', 'CASSVAGGRPTDTQYF')",2,0.524409,0.063776
5,intra,0,total,VTEHDTLLY A0101,607.0,1.670995,"('CAVQPSSGPMNYGGSQGNLIF', 'CASSYTPGSTYEQYF')",2,0.524409,0.063776
...,...,...,...,...,...,...,...,...,...,...
2791701,intra,29,is viable cell,RAKFKQLL B0801,54.0,1.655972,"('CAVSTHGGSQGNLIF', 'CASSITQVGTEAFF')",2,0.504425,0.056093
2791702,intra,29,is viable cell,RAKFKQLL B0801,336.0,1.627353,"('CATVSSDGQKLLF', 'CASSHSRGPDNEQFF')",2,0.504425,0.056093
2791703,intra,29,is viable cell,RAKFKQLL B0801,153.0,1.772556,"('CAYYSGAGSYQLTF', 'CASSADRSEAFF')",2,0.504425,0.056093
2791704,intra,29,is viable cell,RAKFKQLL B0801,373.0,1.537717,"('CAASERGGHTGANSKLTF', 'CASRSTEPNQPQHF')",2,0.504425,0.056093


In [9]:
test_df[test_df.plateau=='inter']

,plateau,rnd_sample,filtering,peptide,ct,score,cdr3s,label,AUC,AUC 0.1
2795100,inter,0,total,VTEHDTLLY A0101,1.0,1.838295,"('CAASQNEKLTF', 'CASRIGAAGNSPLHF')",0,0.524409,0.063776
2795101,inter,0,total,VTEHDTLLY A0101,5.0,1.747185,"('CAVEDRLDNDMRF', 'CSARGLGQAKEYGYTF')",0,0.524409,0.063776
2795102,inter,0,total,VTEHDTLLY A0101,4.0,1.702858,"('CAVTAPAYSGAGSYQLTF', 'CASSYTPGSTYEQYF')",0,0.524409,0.063776
2795104,inter,0,total,VTEHDTLLY A0101,437.0,1.575735,"('CAVTYSPPPKLLF', 'CASSVAGGRPTDTQYF')",0,0.524409,0.063776
2795105,inter,0,total,VTEHDTLLY A0101,607.0,1.523541,"('CAVQPSSGPMNYGGSQGNLIF', 'CASSYTPGSTYEQYF')",0,0.524409,0.063776
...,...,...,...,...,...,...,...,...,...,...
5586801,inter,29,is viable cell,RAKFKQLL B0801,54.0,1.714291,"('CAVSTHGGSQGNLIF', 'CASSITQVGTEAFF')",0,0.504425,0.056093
5586802,inter,29,is viable cell,RAKFKQLL B0801,336.0,1.647721,"('CATVSSDGQKLLF', 'CASSHSRGPDNEQFF')",0,0.504425,0.056093
5586803,inter,29,is viable cell,RAKFKQLL B0801,153.0,1.696678,"('CAYYSGAGSYQLTF', 'CASSADRSEAFF')",0,0.504425,0.056093
5586804,inter,29,is viable cell,RAKFKQLL B0801,373.0,1.573100,"('CAASERGGHTGANSKLTF', 'CASRSTEPNQPQHF')",0,0.504425,0.056093


In [16]:
lst_scores = test_df[test_df.rnd_sample == 0].groupby(['filtering','plateau'],sort=False).score.apply(list).to_frame().reset_index() #,sort=False
lst_scores

,filtering,plateau,score
0,total,intra,"[1.792556, 1.767455, 1.96858, 1.56940500000000..."
1,optimal threshold,intra,"[1.767455, 1.96858, 1.735842, 1.639173, 1.7526..."
2,matching HLA,intra,"[1.767455, 1.720541, 1.72653, 1.548513, 1.8033..."
3,complete TCRs,intra,"[1.792556, 1.767455, 1.96858, 1.56940500000000..."
4,specificity multiplets,intra,"[1.650559, 1.767455, 1.6041600000000005, 1.726..."
5,is cell (GEX),intra,"[1.6694460000000002, 1.767455, 1.6041600000000..."
6,is viable cell,intra,"[1.6694460000000002, 1.767455, 1.96858, 1.5694..."
7,total,inter,"[1.838295, 1.747185, 1.702858, 1.575735, 1.523..."
8,optimal threshold,inter,"[1.747185, 1.579052, 1.59995, 1.620333, 1.6739..."
9,matching HLA,inter,"[1.565142, 1.524096, 1.59995, 1.61866300000000..."


In [17]:
lst_lst_scores = lst_scores.groupby('filtering', sort=False).score.apply(list)
lst_lst_scores

filtering
total                     [[1.792556, 1.767455, 1.96858, 1.5694050000000...
optimal threshold         [[1.767455, 1.96858, 1.735842, 1.639173, 1.752...
matching HLA              [[1.767455, 1.720541, 1.72653, 1.548513, 1.803...
complete TCRs             [[1.792556, 1.767455, 1.96858, 1.5694050000000...
specificity multiplets    [[1.650559, 1.767455, 1.6041600000000005, 1.72...
is cell (GEX)             [[1.6694460000000002, 1.767455, 1.604160000000...
is viable cell            [[1.6694460000000002, 1.767455, 1.96858, 1.569...
Name: score, dtype: object

In [42]:
#lst_scores[(lst_scores.filtering == 'total') & (lst_scores.plateau == 'intra')].score[0]
#lst_scores[(lst_scores.filtering == label) & (lst_scores.plateau == 'inter')].score.values[0]

In [43]:
for label in labels:
    print(label)
    a = lst_scores[(lst_scores.filtering == label) & (lst_scores.plateau == 'intra')].score.values[0]
    b = lst_scores[(lst_scores.filtering == label) & (lst_scores.plateau == 'inter')].score.values[0]
    print(wilcoxon_test(a,b)['test'])

total
True
optimal threshold
True
matching HLA
True
complete TCRs
False
specificity multiplets
True
is cell (GEX)
False
is viable cell
False


In [35]:
b

NameError: name 'b' is not defined

In [19]:
sign_test = lst_lst_scores.apply(lambda x: wilcoxon_test(x[-1],x[0])['test']) #wilcoxon_test()
sign_test

filtering
total                     False
optimal threshold         False
matching HLA              False
complete TCRs             False
specificity multiplets    False
is cell (GEX)             False
is viable cell            False
Name: score, dtype: bool

In [44]:


sign_add = pd.Series([0]*len(labels), index=labels)
print(sign_add)
for s in test_df.rnd_sample.unique():
    lst_scores = test_df[test_df.rnd_sample == s].groupby(['filtering','plateau'],sort=False).score.apply(list).to_frame().reset_index() #
    #lst_scores = test_df.dropna().groupby(['filtering','plateau'],sort=False).score.apply(list).to_frame().reset_index()
    lst_lst_scores = lst_scores.groupby('filtering', sort=False).score.apply(list)
    sign_test = lst_lst_scores.apply(lambda x: wilcoxon_test(x[0],x[-1])['test']) #wilcoxon_test()
    print(sign_test)
    sign_add = sign_add + sign_test
    print(sign_add)

total                     0
optimal threshold         0
matching HLA              0
complete TCRs             0
specificity multiplets    0
is cell (GEX)             0
is viable cell            0
dtype: int64
filtering
total                      True
optimal threshold          True
matching HLA               True
complete TCRs             False
specificity multiplets     True
is cell (GEX)             False
is viable cell            False
Name: score, dtype: bool
total                     1
optimal threshold         1
matching HLA              1
complete TCRs             0
specificity multiplets    1
is cell (GEX)             0
is viable cell            0
dtype: int64
filtering
total                     False
optimal threshold          True
matching HLA               True
complete TCRs             False
specificity multiplets    False
is cell (GEX)             False
is viable cell            False
Name: score, dtype: bool
total                     1
optimal threshold         2
matching

In [50]:
sign_add > len(test_df.rnd_sample.unique())/2

total                     False
optimal threshold          True
matching HLA               True
complete TCRs             False
specificity multiplets     True
is cell (GEX)             False
is viable cell            False
dtype: bool

In [51]:
sign_add

total                     14
optimal threshold         30
matching HLA              30
complete TCRs             11
specificity multiplets    27
is cell (GEX)              0
is viable cell             1
dtype: int64

In [45]:
lst_scores = test_df.groupby(['filtering','plateau'],sort=False).score.apply(list).to_frame().reset_index() #
#lst_scores = test_df.dropna().groupby(['filtering','plateau'],sort=False).score.apply(list).to_frame().reset_index()
lst_lst_scores = lst_scores.groupby('filtering', sort=False).score.apply(list)
sign_test = lst_lst_scores.apply(lambda x: wilcoxon_test(x[0],x[-1])['test']) #wilcoxon_test()
print(sign_test)

filtering
total                     True
optimal threshold         True
matching HLA              True
complete TCRs             True
specificity multiplets    True
is cell (GEX)             True
is viable cell            True
Name: score, dtype: bool


In [57]:
len(lst_lst_scores['is cell (GEX)'][1])

6570

In [58]:
test_df.peptide.unique()

array(['VTEHDTLLY A0101', 'ATTADVDAGSL A0101', 'SVDIHFDPG A0101',
       'RQAYLTNQY A0101', 'TADVDAGSLS A0101', 'IHFDPGFEEQ A0101',
       'EERQAYLTNQY A0101', 'SVDIHFDPGF A0101', 'FSAGAFILYK A0101',
       'NLVPMVATV A0201', 'TNQYMQRMASV A2401', 'TPRVTGGGAM B0702',
       'WPRAPRGQ B0702', 'RAKFKQLL B0801'], dtype=object)